<a href="https://colab.research.google.com/github/carlosmscabral/demo-apigee-genai/blob/main/Apigee_Gemini_Cross_Region_Re_Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apigee - Gemini Cross Region Re-Routing

This sample client content generates a "stress test" towards Gemini in order to reach the per-region rate limit. But the code uses an Apigee API in front of the Gemini endpoints.

Apigee will automatically detect and recover from Vertex/Gemini rate-limit errors and transparently re-route the calls to a backup region.

In [5]:
!gcloud auth application-default login
!gcloud config set project cabral-apigee

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=csmPeddMsIvTEy4PSVtIgBtc5xeUDo&prompt=consent&token_usage=remote&access_type=offline&code_challenge=7W-gPQiL8DjPmFbdbg7XXXoeca3KQgA4TPyTcJUSKpg&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrAAbdQ7mlj1-9lXNRKQxMTLY9mg0cPwzHt7ks4kK_md2cvwsI0yrEYU3krVWXRqPQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Q

In [6]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
import time
from multiprocessing import Pool
from datetime import datetime


def exec_gemini_call(iteration):
    project_id = "cabral-apigee"
    vertexai.init(project=project_id, api_endpoint="https://dev.35.227.240.213.nip.io/active-retry", api_transport='rest')
    model = GenerativeModel("gemini-1.5-pro-001")
    config = GenerationConfig(
        max_output_tokens=100, temperature=0.4, top_p=1, top_k=32
    )
    model.generate_content(
        f"{iteration} - What's a good name for a flower shop that specializes in selling bouquets of dried flowers?", generation_config=config
    )

def main():

    iterations = range(100)

    start_time = time.time()
    print(f" Start time: {datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')}")

    with Pool() as pool:
        _ = pool.map(exec_gemini_call, iterations)

    end_time = time.time()
    print(f" End time: {datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total time elapsed: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

 Start time: 2024-08-02 16:26:11
 End time: 2024-08-02 16:29:07
Total time elapsed: 176.50 seconds
